# Lab | Revisiting Machine Learning Case Study

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('learningSet.csv')
data.head()

/tmp/ipykernel_9224/687616697.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('learningSet.csv')


,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


In [3]:
categoricals = data.select_dtypes(object)

#### Check for null values in all the columns


In [4]:
def get_columns_with_nan_values(data):
    nan_cols = []
    for col in data.columns:
        if data[col].isna().sum() > 0:
            nan_cols.append(col)
    return nan_cols

nan_cols = get_columns_with_nan_values(categoricals)
categoricals[nan_cols].isna().sum()

GEOCODE2    132
dtype: int64

#### Exclude the following variables by looking at the definitions.

Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

* OSOURCE - symbol definitions not provided, too many categories
* ZIP CODE - we are including state already

Identify columns that over 85% missing values

In [5]:
def get_drop_columns(df, threshold=0.85):
    data = df.copy()
    nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>threshold]
    return list(columns_above_threshold['column_name'])

In [6]:
drop_list = ['OSOURCE', 'ZIP'] + get_drop_columns(categoricals, 0.85)
drop_list

['OSOURCE', 'ZIP']

#### Remove those columns from the dataframe

In [7]:
categoricals = categoricals.drop(drop_list, axis=1)
categoricals.columns

Index(['STATE', 'MAILCODE', 'PVASTATE', 'NOEXCH', 'RECINHSE', 'RECP3',
       'RECPGVG', 'RECSWEEP', 'MDMAUD', 'DOMAIN', 'CLUSTER', 'AGEFLAG',
       'HOMEOWNR', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'GENDER',
       'DATASRCE', 'SOLP3', 'SOLIH', 'MAJOR', 'GEOCODE', 'COLLECT1',
       'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO',
       'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER',
       'KIDSTUFF', 'CARDS', 'PLATES', 'LIFESRC', 'PEPSTRFL', 'RFA_2', 'RFA_3',
       'RFA_4', 'RFA_5', 'RFA_6', 'RFA_7', 'RFA_8', 'RFA_9', 'RFA_10',
       'RFA_11', 'RFA_12', 'RFA_13', 'RFA_14', 'RFA_15', 'RFA_16', 'RFA_17',
       'RFA_18', 'RFA_19', 'RFA_20', 'RFA_21', 'RFA_22', 'RFA_23', 'RFA_24',
       'RFA_2R', 'RFA_2A', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A', 'GEOCODE2'],
      dtype='object')

#### Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest.

In [8]:
categoricals['GENDER'].value_counts(dropna=False)

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [9]:
def clean_gender(g):
    if g not in ['F', 'M']:
        return 'other'
    else:
        return g

categoricals['GENDER'] = categoricals['GENDER'].apply(clean_gender)    
categoricals['GENDER'].value_counts(dropna=False)

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64